Рогович Татьяна

# Анализ данных  в Python (политология)

## Предсказываем цену дома Decision Tree и Simple Random Forest

##### Занятия 19-20 | 19.11.2019

Этот блокнот составлен на основе большого блокнота, который доступен по этой [ссылке](https://www.kaggle.com/dansbecker/your-first-machine-learning-model). В нем присутствуют дополнительные комментарии и другие примеры работы алгоритма. Если хотите как следует разобраться с деревьями на базовом уровне - обязательно читайте.

### Загружаем наши данные

Нашим датасетом будет информация о домах, которая включает в себя множество параметров, например, количество и размер этажей дома, год постройки, общая площадь, цена и многое другое (остальные столбцы, можете изучить самостоятельно).

In [1]:
import pandas as pd

#Загружаем данные
home_data = pd.read_csv('https://raw.githubusercontent.com/rogovich/2019-2020_PolSci_Data_Analysis_in_Python/master/12week_ML_Intro/house_data.csv')

### Выбор целевой переменной

В нашем случае, все более-менее очевидно. Это цена на дом. Обычно, цель предсказания помечают y.

In [5]:
y_train = home_data.SalePrice

### Выбор признаков (Features)

Столбцы, которые есть в нашей модели и которые в последствии будут использованы для предсказания, называются признаками (features). В нашем случае, эти колонки будут определять стоимость дома. Иногда используются все колонки, кроме той на которую делается предсказание. А вдругих случаях, лучше выбрать только часть из них. Как определиться? Для этого проводят разведывательный анализ или сравнивают качество работы моделей на разных выборках признаков.

Мы будет использовать не все столбцы, чтобы алгоритм все считал быстрее. Добавим в модель только те столбцы, которые сильнее всего коррелировали с ценой, когда мы исследовали данные.

In [40]:
feature_columns = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'YearRemodAdd']

Обычно, такие данные обозначаются X:

In [41]:
X_train = home_data[feature_columns]

### Строим модель

Мы будем использовать модуль scikit-learn для создания модели. Scikit-learn одна из самых популярных библиотек для моделирования данных, хранящихся в датафреймах. 

Для построения модели нужно выполнить следующие шаги: <br>

Define: Какого типа будет модель? Дерево решений? Какой-то другой тип?<br>
Fit: запускаем модель на тренировочных данных (обучаем ее), чтобы алгоритм нашел в них некие закономерности и зависимости.<br>
Predict: Предсказать результат.<br>
Evaluate: Определить насколько точным оказалось предсказывание, оценить качество модели<br>

В нашем примере, мы будет использовать дерево решений из scikit-learn и тренировать модель с признаками, выделенными ранее. Импортируем функцию DecisionTreeRegressort

In [42]:
from sklearn.tree import DecisionTreeRegressor

Инициализируем модель и обучим ее на наших данных.

In [43]:
# Инициалазируем модель
home_model = DecisionTreeRegressor()

# Обучаем модель
home_model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

Нам выше вывелась информация о нашем регрессоре и с какими парамтерами он обучался.

Мы обучили модель. Но как хорошо она обучилась? Давайте предскажем значения и сравним их с теми ценами, которые на самом деле соответствуют домам.

In [26]:
print("Наше предсказание:", home_model.predict(X_train.tail(10)))
print("Реальная цена домов:", y_train.tail(10).tolist())

Наше предсказание: [136000. 287090. 145000.  84500. 185000. 175000. 210000. 266500. 142125.
 147500.]
Реальная цена домов: [136000, 287090, 145000, 84500, 185000, 175000, 210000, 266500, 142125, 147500]


Результаты похожи, но кое-где есть отличия. И мы отображаем всего 10 элементов из всей выборки. Посмотрим насколько правильно нам предсказала модель для всей выборки. Мы можем, конечно, пройтись в цикле по всем эелементам и посчитать ошибку у каждого. Но у нас могут быть как хорошие, так и плохие предсказания. И в идеальном случае нам нужна одна метрика, которая скажет как хорошо мы обучили данные.

Воспульзуемся метрикой MEA (Mean Absolute Error) - Cредняя Aбсолютная Ошибка. Ее можно представить в таком виде: ошибка = реальная цена − предсказанная цена. Например, если цена дома 150 000, мы предсказали цены в 100 000, то ошибка будет 50 000. Для ошибок в отрицательную сторону берем модуль. Это одна из простейших метрик.

Импортируем нужную функцию опять же из sklearn

In [27]:
from sklearn.metrics import mean_absolute_error

Делаем предсказание и передаем в функцию предсказанное и реальное значение

In [28]:
pred = home_model.predict(X_train)
mae = mean_absolute_error(pred, y_train)
print(mae)

111.4013698630137


Получили ошибку достаточно маленькую ошибку цены в долларах, что было бы очень хорошим результатом, если не одно но.

### Валидация данных

Мы только что проверяли обученную модель на тех же данных, на которых мы ее и обучали. И это неверно. Т.к. в нашем случае нам нужно предсказывать цены новых домов, которых не было в обучающей выборке. Поэтому не удивительно, что MAE = 111. 

Опишем эту проблему таким примером. У нас есть большой рынок жилья. Цена жилья не связана с цветом входной двери. Но в нашей обучающей выборке у всех домов с высокой ценой была зеленая дверь. Модель увидит эту зависимость и будет предсказаывать для новых домов с зеленой дверью высокую цену, что не будет корректным. Но при валидации модели на обучающих данных мы получили маленькую ошибку и посчитали, что модель натренирована хорошо. Если бы мы это сделали на тестовых данных, то увидели бы что ошибка велика и нужно переделать модель.

### Обучающие и тестовые выборки

Давайте разделим наши данные на 2 части: обучающую выборку и тестовую. Воспльзуемся опять модулем sklearn и функцией train_test_split.

In [29]:
from sklearn.model_selection import train_test_split

Разобъем данные как для X, так и для  y. Каждый раз разбитие происходит случайно, мы можем только указать в какой пропорции мы хотим разбить наши данные, например 30% всей выборки на обучение, остальное на тест.

In [58]:
# заполнили массивы с нашими данными 
X = home_data[feature_columns]
y = home_data.SalePrice

# Делим
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 42, test_size = 0.25)

Посмотрим сколько данных попала в обучающую и тестовые выборки

In [59]:
print(len(train_X))

1095


In [60]:
print(len(test_X))

365


В обучающей 1095, а в тестовой 365 домов.

Теперь как и в прошлый раз построим модель и посчитаем MAE

In [63]:
# Инициализируем модель
home_model = DecisionTreeRegressor()
# Обучаем модель
home_model.fit(train_X, train_y)

# Предсказываем цены и считаем MAE
val_predictions = home_model.predict(test_X)
print(mean_absolute_error(test_y, val_predictions))


26182.909589041097


Воу. Это большая ошибка по сравнению с нашей изначальной ошибкой на обучающей выборке. И выходит, что для новых домов, наша модель плохо предсказывала бы цены

Наша модель обучилась не очень оптимально, но к счастью есть несколько способов как можно улучшить модель, например, экспериментируя с параметрами модели.

### Эксперементируем с моделями

Можно сказать что мы столкнулись с проблемой переобучения (overfitting), когда модель предсказывает обучающие данные практически идеально, а тестовые данные и другие данные предсказывате плохо.

Существует еще и недообучение (underfitting), когда модель не может найти параметры по которым можно хорошо разделить данные, и она плохо предсказывает даже на обучающих данных.

Так как мы используем DecisionTreeRegressor, т.е. дерево, в случае overfitting дерево обычно очень глубокое, а при underfitting получаем деревья небольшие. Поэтому мы может попробовать регулировать глубину дерева. В DecisionTreeRegressor есть параметер max_leaf_nodes, который мы можем контролировать. Чем больше листьев мы разрешим модели делать, тем дальше мы уйдем от underfiting, но будем приближаться к overfitting, т.е. в идеальном случае нам нужно подобрать примерно среднюю глубину дерева для модели.

Создадим функцию, которая будет строить для нас деревеья с различным параметром max_leaf_nodes. А замерять модели будем снова через MAE.

In [64]:
def get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y):
    # инициализируем модель с параметром max_leaf_node, который мы передали в функцию
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    # обучаем модель на обучающих данных
    model.fit(train_X, train_y)
    # предсказываем на тестовых данных
    preds_val = model.predict(test_X)
    # считаем MAE
    mae = mean_absolute_error(test_y, preds_val)
    # возвращем значение ошибки
    return(mae)

В функцию мы соответственно передаем количество листов, которые мы хотим и обучающие и тестовые выборки. Сделаем небольшой цикл по различный значением глубины от 5 до 5000 и выведем результаты.

In [65]:
# сравниваем различне значения MAE для различной глубины деревьев
for max_leaf_nodes in [5, 10, 50, 100, 500, 1000, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y)
    print("Максимальная глубина дерева: %d  \t\t MEA:  %d" %(max_leaf_nodes, my_mae))

Максимальная глубина дерева: 5  		 MEA:  31239
Максимальная глубина дерева: 10  		 MEA:  28860
Максимальная глубина дерева: 50  		 MEA:  23791
Максимальная глубина дерева: 100  		 MEA:  23124
Максимальная глубина дерева: 500  		 MEA:  24609
Максимальная глубина дерева: 1000  		 MEA:  25065
Максимальная глубина дерева: 5000  		 MEA:  25069


Как мы видим, самый лучший результат мы получили для глубины 100, но при этом ошибка достаточно большая и искажает предсказания для новых данных. С другой стороны, если мы посмотрим описательные статистики для нашей цены, мы увидим, что по сравнению со медианой и интерквартильным размахом, наша ошибка не настолько ужасна. Поэтому величина ошибки предсказания достаточно хитрая штука для интерпретации.

In [66]:
train_y.describe()

count      1095.000000
mean     181712.286758
std       77955.082565
min       34900.000000
25%      130000.000000
50%      165000.000000
75%      215000.000000
max      745000.000000
Name: SalePrice, dtype: float64

### Random Forest

Дерево решений оставляет с серьезным выбором. Глубокое дерево с overfitting с несколькими домами в каждом листе или мелкое дерево с несколькими листами, которое не может определить параметры и разделить данные? И все это с приличной ошибкой.

Некоторые другие модели деревьев смогли решить эту проблему и уменьшить при этом итоговую ошибку. Попробуем взять Random Forest. Он использует множество деревьев и делает предсказание путем усреднения прогнозов каждого дерева.

[Подробнее про случайный лес тут.](https://dyakonov.org/2016/11/14/%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D0%B9-%D0%BB%D0%B5%D1%81-random-forest/)

Попробуем использовать Random Forest на наших данных. Импортируем его из модуля sklearn

In [67]:
from sklearn.ensemble import RandomForestRegressor

Строим теперь модель с тем же параметрами и данными как в DecissionTreeRegressor

In [70]:
# Инициализируем модель
forest_model = RandomForestRegressor(random_state=1)
# Обучаем модель
forest_model.fit(train_X, train_y)
# Делаем предсказание
randomf_preds = forest_model.predict(test_X)
# Посчитаем и выведем MAE
print(mean_absolute_error(test_y, randomf_preds))

20341.379141552512


Ошибка уменьшилась, но все равно составляет больше 20 тысяч долларов. Можно продолжить дальше эксперементировать с моделью, изменяя параметры или увеличивая обучающую выборку. 

Давайте изменим параметр n_estimators, который отвечает за количество деревьев в модели ( не путать с количеством листьев в DecisionTree)

In [69]:
# Инициализируем модель
forest_model = RandomForestRegressor(n_estimators = 100, random_state=1)
# Обучаем модель
forest_model.fit(train_X, train_y)
# Делаем предсказание
randomf_preds = forest_model.predict(test_X)
# Посчитаем и выведем MAE
print(mean_absolute_error(test_y, randomf_preds))

19141.684105175038


Уже 19 тысяч! Много это или мало? Можно ли остановиться? 

Конечно, мы всегда стараемся минимизировать нашу ошибку, но это и не всегда возможно. Возможно, здесь лучше справится другая модель. Так же мы взяли очень мало признаков, при чем не основываясь на какой-то особой теории - возможно, преобразование наших признаков или же их замена, улучшили бы качество.

С остальными параметрами можно ознакомиться в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) функции